# Assignment 2

**Handin in Peergrade**: *Friday*, November 15, 2019, 23:59<br>
**Peergrading deadline**: *Monday*, November 18, 2019, 23:59<br>
**Peergrading feedback deadline**: *Friday*, November 22, 2019, 23:59

[**Feedback**](http://ulfaslak.com/vent)

## Problems

### Week 5

In [8]:
%matplotlib inline

import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop

In [9]:
import requests as rq
response = rq.get("http://www.dailyscript.com/scripts/pulp_fiction.html")
pulpfiction = BeautifulSoup(response.content, "html.parser").getText()
#print(pulpfiction[:2000])

In [4]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# char_indices and indices_char order the characters in the text by alphabetical order
# char_indices has the the char's number in order as the value while indices_chars has the number in order as the key
chars = sorted(list(set(pulpfiction)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
#seqlen is used to take a set of (in this instance) 40 chars from the text. It is used it shrink the range
#of the for loop to avoid an out of range error, and is used as the step size for range so i is indexed once over
#every seqlen characters 
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(pulpfiction) - seqlen - 1, step):
    sentences.append(pulpfiction[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(pulpfiction) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = pulpfiction[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
7614/7614 [==============================] - 7s 893us/step - loss: 2.2057 - categorical_crossentropy: 2.2057 - accuracy: 0.5422

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "                      THE WOLF
        "
                      THE WOLF
                                                                                                                                                                                                                                                                                                                                                                                                                        
----- diversity: 0.5
----- Generating with seed: "                      THE WOLF
        "
                      THE WOLF
                                                                            

                                                                                                                                                                                                                                                                                                                                                                                                                         
----- diversity: 1.0
----- Generating with seed: "   and have a heart attack.

         "
   and have a heart attack.

            ?
             Curlluba.

                                               I'll a doeg, a nlioghtaif syere.

                                                  Appery on Rimes it torera.

                 (veresed. Andyl... she, thase I'm fucker, 
                                                  JULLS
 "                                     Evim.intresta trucks. Coom forge.

           
Epoch 6/50
7614/7614 [==============================] - 7s 966us/step - 

        Same motel room, except empty. When he 
                                                                                                                                                                                                                                                                                                                                                                                                       
----- diversity: 0.5
----- Generating with seed: "        Same motel room, except empty. W"
        Same motel room, except empty. What Marsellus 
               here friend with a back and we fell 
                                                         JULES
                                                                                                                                    VINCENT
                                                                                                                              
----- diversity: 1.0
----- Gene

                                       (wat's got ary ox forran. So want open?

                                     Exmerenty. Bewrro.

                         the foow.

                         Pumpkin' for back, wat' knugger and Put 
               your croint of the Fabienne for it under does kigh a wamp 
               mouthing Wold arrinQt. Loin'. AlleL

           
Epoch 15/50
7614/7614 [==============================] - 7s 911us/step - loss: 0.7836 - categorical_crossentropy: 0.7836 - accuracy: 0.7812

----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "                   you, this has been wi"
                   you, this has been with the rest of 
                                                                                                                                                                                                                                                                                                        

                         Well you think was a magad it to his back 
                         what's it ain't looks line his head: "Yes."

                                                                                                                                     JULES
                         What was the fucking out of the 
                         factin' did you never to it.

              
----- diversity: 1.0
----- Generating with seed: "                  YOUNG MAN
           "
                  YOUNG MAN
                         THE WOLF
                         Back if their don't bathenion who

                         the phone's TRENTER CARCELS." down on the drivar:

               INT. MARSELLUS WOMANS t'd fidddring 
               "EDy, "Day beed down the husse.

                              across neverd or the just beddery 
                       was and from the madal! Tave 
             
Epoch 20/50
7614/7614 [==============================] - 7s 900us/step - l

...

                                     Butch?

                                     Well me Marsellus coupless, Jules sticks apprancejs in-tonere her anymirrot fathrobe 
               can't saying his not defitality?

                                     If you know somethin'. Jules and Vincent vercaushing dout behind
Epoch 29/50
7614/7614 [==============================] - 7s 890us/step - loss: 0.6837 - categorical_crossentropy: 0.6837 - accuracy: 0.8068

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: " cup of coffee.

                     "
 cup of coffee.

                                                                                                                                                                                                                                                                                                                                                                                                           

                         gun let'le got meven they had go to 
                         good go the between looking hand house her 
                         the fingers in the race of Jules and shaking in the body stuckins 
                         you got a bobbye, Butch stands inside, 
                         a cobboor checause bed, three's a fact 
                         better.

          
----- diversity: 1.0
----- Generating with seed: "rs. The BUSBOYS going through the 
    "
rs. The BUSBOYS going through the 
               the shot, pulled home in a DINIP makes 
               chary.

                                          Ishides his front of the wall.

                                     FABIENNE
                         goes?

               .. Jully.

                                               JULES
                         Hel 'sm-ted muckost table in skilled to less out a down Vin
Epoch 34/50
7614/7614 [==============================] - 7s 955us/step - loss: 0.6

               has starring to the room, the back and 
                         the billet in a hellow to the fingers 
                         the banksed in his little man arrosse 
                         the banks.

                                                                                                                                                                                                    
----- diversity: 0.5
----- Generating with seed: "ng lot. The rain 
               has st"
ng lot. The rain 
               has stands up a menutell's somethin' 
                         we little me fine, watch the wall.

                                                                             YOUNG WOMAN
                         He drives to himself, that's what I'm 
                         what you it of minues his little line, 
                         parther abodd's paid a foot massage 
                    
----- diversity: 1.0
----- Generating with seed: "ng lot. T

                                          with Marsellus Wallace like this 
                         takes as We laccle my marty PANCE, 
                         its because storting Pumpuing, 
                         yourtall menure my finder a doubonglate 
                         and by filtwenopry hay black in Jim, into the 
      
Epoch 43/50
7614/7614 [==============================] - 6s 832us/step - loss: 0.6453 - categorical_crossentropy: 0.6453 - accuracy: 0.8172

----- Generating text after Epoch: 42
----- diversity: 0.2
----- Generating with seed: "K AND FORTH during the 
               "
K AND FORTH during the 
                         to say you get it up.

                                                                                                                                                                                                                                                                                                                               

                         standle may.

               INT. MARCELLUS' HOUSE / DRESS
                         What'd they coold bockes in the face I know, 
                         the show one window, it and good mick in 
                         the silence?

                                              FABIENNE
                         It's out window.

                                   
----- diversity: 1.0
----- Generating with seed: "o the motel parking lot. The rain 
    "
o the motel parking lot. The rain 
                         fast of his apsalkin' Fuck it.

                                     BRETT
                         Butch still executtion day Choffiouars back on ass who, nobigors parked with 
                         mind in a staybes, Butch Collealival armact?  a werb to be 
                                        Whatgucle up tomato, he's gone. We seen.

               Reskeca here
Epoch 48/50
7614/7614 [==============================] - 8s 1ms/step - loss: 0.63

> **Ex. 5.1.4**: Did the network finish training? Consider the generated text across epochs.
1. In the early batches (0-10), the generated text looks very bad. Can you explain why the low diversity generated text contains almost only the symbol " " (that is, spaces)?
2. The high diversity generated text is messed up too, but in a different way. Explain how.
3. In later batches (20-30) what do you notice is off about the low diversity generated text?

<div class="alert alert-block alert-info">
<b>ANS 5.1.4.1 </b>   
    Diversity is a measure of "temperature" in an LSTM model. It controls randomness of predicted values by scaling the input (dividing the input by the diversity value) before using the softmax function. This makes the results more confident and more conservative for the low diversity generated text when using softmax, sticking to the most likely outputs. In this case, that is spaces as we saw in exercise 5.1.1.
    
<b>ANS 5.1.4.2 </b> 
The high diversity generated text is messed up because increased diversity scales the input less than low diversity. This leads to both less confident outputs and more randomness and hence more erratic spelling mistakes and output that doesn't resemble English words.

<b>ANS 5.1.4.3 </b>
The low diversity text becomes very redundant. It repeats many of the same phrases. Example from epoch 36 one of the times we ran it:


        It ain't a date. It's like when you eat 
                             the showerfuckers the three through the 
                             the shot. He's not now who has the street to 
                             the showerfuckers the through the corter in the corner 
                             the show, no your nothin' she gun snow 
                             the throwes one show to the exits them door.

</div>



In [10]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# char_indices and indices_char order the characters in the text by alphabetical order
# char_indices has the the char's number in order as the value while indices_chars has the number in order as the key
pulpfiction = pulpfiction[:5000]

chars = sorted(list(set(pulpfiction)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
#seqlen is used to take a set of (in this instance) 40 chars from the text. It is used it shrink the range
#of the for loop to avoid an out of range error, and is used as the step size for range so i is indexed once over
#every seqlen characters 
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(pulpfiction) - seqlen - 1, step):
    sentences.append(pulpfiction[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(pulpfiction) - seqlen - 1)
    
    # Q5: What does diversity do?
    diversity = 0.5
    print('----- diversity:', diversity)

    generated = ''
    sentence = "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
124/124 [==============================] - 0s 4ms/step - loss: 4.1726 - categorical_crossentropy: 4.1726 - accuracy: 0.0097

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg                                                                                                                                                                                                                                                                                                                                                                                                                
Epoch 2/50
124/124 [==============================] - 0s 931us/step - loss: 3.5054 - categorical_crossentropy: 3.5054 - accuracy: 0.5135

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
wnfkhjeosncfghjkkjhgfydtrfy

                                                                                                                             a                                                         
r                                                     
Epoch 13/50
124/124 [==============================] - 0s 838us/step - loss: 2.3998 - categorical_crossentropy: 2.3998 - accuracy: 0.5141

----- Generating text after Epoch: 12
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
 nfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg                                                                                                                                                  n                                                                   e         y                                                 
 e                                                                                                                     
Epoch 14/50
124/124 [==============================] - 0s

 nfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfgeg                         e                              a                      a                         h           r                             os a                                                              e  ae               r a     a.      t                               a    e  a    s  o       l  a                 t     os    t        a                               i            
Epoch 25/50
124/124 [==============================] - 0s 817us/step - loss: 2.3065 - categorical_crossentropy: 2.3065 - accuracy: 0.5143

----- Generating text after Epoch: 24
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg                                                                                                                                                                                                                                                                

                                                                                                               
Epoch 36/50
124/124 [==============================] - 0s 823us/step - loss: 2.2295 - categorical_crossentropy: 2.2295 - accuracy: 0.5383

----- Generating text after Epoch: 35
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg            otog                  o                              e          t                 '                                                                          ne                                                                                      W                                  t                 g         a                                                i a                            
Epoch 37/50
124/124 [==============================] - 0s 790us/step - loss: 2.1189 - categorical_crossentropy: 2.1189 - accuracy: 0.5159

----- Generating text after Epoch: 36
-

wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg                                                                                                                                                                                                                                                                                                                                                                                                                
Epoch 48/50
124/124 [==============================] - 0s 811us/step - loss: 2.2679 - categorical_crossentropy: 2.2679 - accuracy: 0.5333

----- Generating text after Epoch: 47
----- diversity: 0.5
----- Generating with seed: "wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfg"
wnfkhjeosncfghjkkjhgfydtrfyguhiytrsrdtfgan


                              po                                                            














                         
                                                                                                                

> **Ex. 5.1.6**: Do the same as above, but for 40 random letters (e.g. smash away on your keyboard) as seed. What happens? Can you explain why?

<div class="alert alert-block alert-info">
<b>ANS 5.1.6 </b> 
Accuracy reaches 55.56% which is just above the baseline accuracy if the model were to predict just the most common symbol (space). This is because this sequence of 40 characters does not appear anywhere in the text so the seed has no relevance to what the model is trying to fit and so it is stuck guessing randomly.
    
The model's output after the first epoch is just the gibberish seed and after the 50th epoch it is the seed with spaces and one character 't'. In between, the model outputs different combinations of random letters and a lot of spaces. The models with a lot of spaces consistently score higher accuracy but with greater loss (categorical cross entropy)  
</div>

### Week 6

> **Ex. 6.1.1**: In your own words, explain what the following function arguments do in
the different model loading functions:
1. `include_top`
1. `weights`
1. `input_shape`
1. `pooling`
1. `classes`
1. Explain what 'global pooling' does, and why it is needed when `include_top=False`

<div class="alert alert-block alert-info">
<b>ANS 4.3.2 </b>   
    
1. `include_top` - if include_top is set to false the fully-connected output layers that makes predictions is not loaded which allows a new output layer to be added and trained by using the argument input_tensor.   
    
1. `weights` - allows you to choose between pretrained weights or initialize the model with random weights by setting the argument to None (random initialization) or 'imagenet' (pre-training on ImageNet). 

1. `input_shape` - needs to be specified when include_top is False if not the default shape will be (224,224,3), input shape must have 3 input channels and the width and height can't be smaller than 32. 

1. `pooling` - can be used for models with include_top False. There can be avg or global max pooling. The result of pooling is a vecotr that can be used as a feature descriptor. When pooling is set to None the output of the model will be the 4D tensor from last convolutional block.'avg' means the output of the model will be a 2D tensor. 'max' means that global max pooling will be applied.  

1. `classes` - defines the number of classes for model to classify images from dataset into, only specify is when include_top is True and when no weights argument is provided, the argument will then be configured in the output layer
    
1. Explain what 'global pooling' does, and why it is needed when include_top=False  
It creates a vector that can be used as a feature descriptor for an input. When include_top=Flase the fully connected output layers are not loaded, therefore the model needs a convolutional or pooling layer to ouput activations. 

</div>

> **Ex. 6.1.2**: Following Jason's example under 'Pre-Trained Model as Classifier'
classify [this image](https://66.media.tumblr.com/tumblr_mc46e7Zm4R1qbqngeo1_1280.jpg).
Print not just the most likely label, but everything that `decode_predictions` returns.
>
> ***Important***: *Don't use VGG as he does. It's 500 MB to download, and will take too long.
> Use one of the smaller models instead ([here](https://keras.io/applications/#documentation-for-individual-models)'s an overview of model sizes), such as DenseNet121.*

In [8]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = '/Users/alderik/Documents/School/College/DIS/annadl_f19/exercises/Ulf.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds)[0])

Predicted: [('n03000684', 'chain_saw', 0.055904984), ('n03534580', 'hoopskirt', 0.048483256), ('n02999410', 'chain', 0.046680905), ('n03110669', 'cornet', 0.041602883), ('n03763968', 'military_uniform', 0.035697747)]


### Week 7

> **Ex. 7.1.1**: What is typically the input and output of an autoencoder? What loss function can be used?

<div class="alert alert-block alert-info">
<b>ANS 7.1.1 </b>   
    
The input is typically something with a very high dimensionality (image, vector, etc.). The output is a smaller representation (less dimensions) than the input. By comparing pixel to pixel differences in the output a loss function can be created. 
</div>

> **Ex. 7.1.3**: Purely in terms of architecture, what is the difference between an autoencoder and a variational autoencoder (VAE)?

<div class="alert alert-block alert-info">
<b>ANS 7.1.3 </b>   
    
The normal bottleneck vector is replaced in a VAE. There are instead two vectors: one representing the mean of the distribution and one representing the standard deviation. 
</div>


> **Ex. 7.1.4**: Regular autoencoders are trained to minimize a loss function with no regard to how the latent space is organized. Therefore, continuity is not guaranteed and similar datapoints may not be close to each other. We can thus say that the network is overfitting, because it uses any organization of training points in this space to minimize the loss, and is, therefore, not likely to work well with unseen data. VAEs are a regularized form of autoencoders, invented to solve this problem. Importantly, they guarantee that similar points are close in the latent space. How do they achieve this?
    > * How are datapoints represented in the VAE latent space? What is the intuition behind this?
    > * How is the loss function different? What is the purpose of the second term (the KL divergence)?
>
> *Hint: Check out this [blog post](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73) and read the section "Intuitions about the regularisation"*

<div class="alert alert-block alert-info">
<b>ANS 7.1.4 </b>   
    
- The datapoints are represented as distributions instead of simple points. This is because in order to make generative process possible, two properties must be expressed: continuity (two close points in the latent space should not give two completely different contents once decoded) and completeness (for a chosen distribution, a point sampled from the latent space should give “meaningful” content once decoded).
- The loss function consists of two terms. The purpose of the second term is to make sure that the distribution being learned is not too far removed from a normally distributed discussion. (trying to force laten distribution to be close to a mean of 0 and a standard deviation of 1)
</div>


> **Ex. 7.2.1**: Explain in your own words how the GAN works. Touch upon:
    > * What do the generator and discriminator networks do?
    > * What are their respective input and output?
    > * What would the accuracy of the discriminator be, faced with a perfect generator?

<div class="alert alert-block alert-info">
<b>ANS 7.2.1 </b>   
    
The generator takes in a random noise vector and ends up outputing an image by going through a bunch of convalational layers. The discrimminators one job is to tell if an image is real (from the datatset), or fake (from the generator). A perfect generator would lead to a discrimminator that would always be fooled. In a perfect world you want them to be balanced. 
</div>


> **Ex. 7.2.4**: How do you transform one image to another using backprop and
gradient descent? Why does this not always work that well? How is transfer learning
used to make it work?

<div class="alert alert-block alert-info">
<b>ANS 7.2.4 </b>   
    
You can find the pixel by pixel difference and then you send your gradients through the generator and update the latent vector Z at the beginning of the generator. This does not work that well because it will reach a local minimum (picture looks very off from what you want) and the optimization objective can't get out. Transfer learning can be used by using a pretrained vgg-network to that will extract a feature vector that will give us a high level representation of what is in the image. 
</div>

